Your Student ID: 67011352
Your full name: Theepakorn Phayonrat

What you need to do:

1.   Make a copy of the above colab notebook.
2.   Write your student ID and your full name in the cell above.
3.   Set the permission of your notebook copy so that anyone with the link can read.
4.   Complete the implementation of the functions multiply_karatsuba and matmul_strassen. Test that your implementation is correct.
5.   Find the least n such that multiply_karatsuba is faster than multiply_school. Write your answer in your colab notebook
6.   Find the least n such that matmul_stressen is faster than matmul_school. Write your answer in your colab notebook.
7.   Save and download this notebook as an ipynb file and submit the file on GoEDU.

# Integer Multiplication

In [90]:
import math

In [91]:
def multiply_school(x,y):
    result = 0
    position = 0

    while y > 0:
        # Check the least significant bit of y
        if y & 1:  # If the current bit is 1, add the shifted x to the result
            result += x << position
        # Shift y to the right (divide by 2) to process the next bit
        y >>= 1
        # Move to the next bit position
        position += 1
		
    return result

In [92]:
def multiply_karatsuba(x,y,n):
	result = 0
	# Your code here
	m = math.ceil(n/2)
	a = x >> m
	b = x % 2**m
	c = y >> m
	d = y % 2**m
	i = (a * c)
	j = (b * d)
	k = (i + j - (a - b) * (c - d))
	return (i << 2**(2*m)) + (k << 2**m) + j

In [93]:
import random
n = 64
x = random.randint(2**n,2**(n+1))
y = random.randint(2**n,2**(n+1))
print("x =", x)
print("y =", y)

x = 18637281131587884017
y = 32847942799415740953


In [94]:
%timeit -n 1000 multiply_school(x,y)

5.76 μs ± 339 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [95]:
%timeit -n 1000 multiply_karatsuba(x,y,n)

MemoryError: 

What is the least value of n such that Karatsuba's algorithm is faster than the grade school algorithm?

# Matrix Multiplication

In [ ]:
def matmul_school(A,B):
	n = len(A)
	C = [[0 for _ in range(n)] for _ in range(n)]
	for i in range(n):
		for j in range(n):
			for k in range(n):
				C[i][j] += A[i][k] * B[k][j]
	return C


In [ ]:
def split_mat(M):
    n = len(M)
    m = n // 2
    M11 = [row[:m] for row in M[:m]]
    M12 = [row[m:] for row in M[:m]]
    M21 = [row[:m] for row in M[m:]]
    M22 = [row[m:] for row in M[m:]]
    return (M11, M12, M21, M22)

In [ ]:
def combine_mat(C11, C12, C21, C22):
    if len(C11) == 1:
        return [[C11[0][0], C12[0][0]],
                [C21[0][0], C22[0][0]]]

    n = len(C11)
    m = n // 2

    top = combine_mat(
        C11[:len(C11)//2],
        C12[:len(C12)//2],
        C21[:len(C21)//2],
        C22[:len(C22)//2]
    )

    bottom = combine_mat(
        C11[len(C11)//2:],
        C12[len(C12)//2:],
        C21[len(C21)//2:],
        C22[len(C22)//2:]
    )


    # top = combine_mat(
    #     C11[:m], C12[:m],
    #     C21[:m], C22[:m]
    # )

    # bottom = combine_mat(
    #     C11[m:], C12[m:],
    #     C21[m:], C22[m:]
    # )

    return top + bottom


In [ ]:
def add(A, B):
    if len(A) == 1:
        return [[A[0][0] + B[0][0]]]

    # n = len(A)
    # m = n // 2

    A11, A12, A21, A22 = split_mat(A)
    B11, B12, B21, B22 = split_mat(B)

    return combine_mat(
        add(A11, B11),
        add(A12, B12),
        add(A21, B21),
        add(A22, B22)
    )


In [ ]:
def sub(A, B):
    if len(A) == 1:
        return [[A[0][0] - B[0][0]]]

    # n = len(A)
    # m = n // 2

    A11, A12, A21, A22 = split_mat(A)
    B11, B12, B21, B22 = split_mat(B)

    return combine_mat(
        sub(A11, B11),
        sub(A12, B12),
        sub(A21, B21),
        sub(A22, B22)
    )


In [ ]:
def matmul_strassen(A,B):
    if len(A) == 1:
        return [[A[0][0] * B[0][0]]]

    A11, A12, A21, A22 = split_mat(A)
    B11, B12, B21, B22 = split_mat(B)

    M1 = matmul_strassen(add(A11, A22), add(B11, B22))
    M2 = matmul_strassen(add(A21, A22), B11)
    M3 = matmul_strassen(A11, sub(B12, B22))
    M4 = matmul_strassen(A22, sub(B21, B11))
    M5 = matmul_strassen(add(A11, A12), B22)
    M6 = matmul_strassen(sub(A21, A11), add(B11, B12))
    M7 = matmul_strassen(sub(A12, A22), add(B21, B22))

    C11 = add(sub(add(M1, M4), M5), M7)
    C12 = add(M3, M5)
    C21 = add(M2, M4)
    C22 = add(sub(add(M1, M3), M2), M6)

    return combine_mat(C11, C12, C21, C22)


In [ ]:
import random
n = 10
A = [[random.randint(-10,10) for _ in range(n)] for _ in range(n)]
B = [[random.randint(-10,10) for _ in range(n)] for _ in range(n)]
print("A =", A)
print("B =", B)

A = [[10, -8, -5, 6, 2, 5, -4, -2, 1, -1], [-8, 7, -9, 3, 0, 2, 7, -3, 4, 5], [-6, -3, 7, 0, 4, -10, -1, 2, -9, 2], [9, -2, -8, 3, 9, -8, 4, 4, 0, 6], [-6, -6, 2, -10, 9, 4, 4, 8, 0, 10], [-7, -4, 4, 7, -6, 0, 10, 4, 9, 2], [-10, 2, 7, 2, -2, 7, 9, -2, 2, 10], [8, -5, 1, -10, 0, -8, 6, -1, -5, 10], [-6, -9, 9, 7, -4, 8, 2, 1, -8, 2], [3, 8, -6, 5, 1, 5, 1, -2, 8, 6]]
B = [[5, 4, 1, -9, 8, 8, -10, -8, 3, -10], [9, 6, 7, 10, 6, 5, 7, 4, -1, -9], [4, -5, -6, -5, -10, -8, -1, 7, 1, 7], [8, 4, -6, -8, 1, 2, 6, 4, -2, 1], [3, -9, 2, -9, -10, -4, -6, 5, -4, -2], [-8, -4, -8, -2, 1, 2, 10, 9, -8, -2], [10, -2, -9, -3, -9, -7, 6, -1, 8, -5], [-5, 5, -8, -7, -10, -2, 4, -3, -7, -10], [-10, 4, -5, -1, -4, -8, -6, -6, -7, -7], [8, -10, 9, 10, 9, -8, -4, -4, 3, -5]]


In [ ]:
%timeit -n 1000 matmul_school(A,B)

67.7 μs ± 1.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit -n 1000 matmul_strassen(A,B)

IndexError: list index out of range

What is the least value of n such that Strassen's algorithm is faster than the grade school algorithm?